# Thresholds on the avg correlation

In [222]:
#Conda env: rpy2-env
import pandas as pd
import numpy as np
import scanpy as sc
import sys
import os
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

# Load the LaplacesDemon R package
robjects.r('library(LaplacesDemon)')

'Laplaces...,'tools','stats',...,'datasets','methods','base'


# Define DoubleMAD function

In [223]:
# DoubleMAD function
def DoubleMAD(x, zero_mad_action="warn"):
    """
    The zero_mad_action determines the action in the event of an MAD of zero.
    Possible values: "stop", "warn", "na" and "warn and na".
    """
    x = np.array(x)
    x = x[~np.isnan(x)]
    m = np.median(x)
    abs_dev = np.abs(x - m)
    left_mad = np.median(abs_dev[x <= m])
    right_mad = np.median(abs_dev[x >= m])
    if left_mad == 0 or right_mad == 0:
        if zero_mad_action == "stop":
            raise ValueError("MAD is 0")
        if zero_mad_action in ["warn", "warn and na"]:
            warnings.warn("MAD is 0")
        if zero_mad_action in ["na", "warn and na"]:
            if left_mad == 0:
                left_mad = np.nan
            if right_mad == 0:
                right_mad = np.nan
    return left_mad, right_mad

# load Data

In [224]:
# Set mouse id
mouse = '669324'

# set directory and filename
input_dir = '/allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/remimathieu/MERSCOPES/development/clustering/analysis'
input_dir = os.path.join(input_dir,'mouse_'+ mouse)

In [225]:
#Load adata object
adata= sc.read_h5ad(os.path.join(input_dir, 'mouse_'+mouse+'_combined_fullpreprocessed_CDMv2.h5ad'))
adata

/allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/remimathieu/miniconda3/envs/rpy2-env/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 5361638 × 550
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x', 'max_y', 'barcodeCount', 'corrected_x', 'corrected_y', 'origin', 'rotation', 'section', 'n_genes_by_counts', 'total_counts', 'total_counts_Blank', 'pct_counts_Blank', 'total_counts_per_cell_volume', 'total_counts_Blank_per_cell_volume', 'n_genes', 'blank_qc', 'doublets', 'Solo_prediction', 'leiden', 'DoubletFinder_prediction', 'CDM_class_label', 'CDM_class_name', 'CDM_class_bootstrapping_probability', 'CDM_class_avg_correlation', 'CDM_subclass_label', 'CDM_subclass_name', 'CDM_subclass_bootstrapping_probability', 'CDM_subclass_avg_correlation', 'CDM_supertype_label', 'CDM_supertype_name', 'CDM_supertype_bootstrapping_probability', 'CDM_supertype_avg_correlation', 'CDM_cluster_label', 'CDM_cluster_name', 'CDM_cluster_alias', 'CDM_cluster_bootstrapping_probability', 'CDM_cluster_avg_correlation', 'min_corrected_x', 'min_corrected_y', 'spatial_x', 'spatial_y', 'o

In [226]:
# extract metadata
metadata = adata.obs

#delete adata to save memory
del adata

In [227]:
# Converts categories in str
metadata['CDM_class_name']= metadata['CDM_class_name'].astype(str)
metadata['CDM_subclass_name']= metadata['CDM_subclass_name'].astype(str)
metadata['CDM_supertype_name'] = metadata['CDM_supertype_name'].astype(str)
metadata['CDM_cluster_name'] = metadata['CDM_cluster_name'].astype(str)

# Define cluster_df

In [230]:
# Dataframe with cluster, supertype, subclass and class

cluster_df = metadata[["CDM_class_name","CDM_class_color","CDM_subclass_name", "CDM_subclass_color", "CDM_supertype_name", "CDM_supertype_color", "CDM_cluster_name", "CDM_cluster_color"]].drop_duplicates()

#Assign id for each hierarchy
cluster_df["CDM_cluster_id"]  = cluster_df["CDM_cluster_name"].str.split(" ").str[0]
cluster_df["CDM_subclass_id"]  = cluster_df["CDM_subclass_name"].str.split(" ").str[0]
cluster_df["CDM_supertype_id"]  = cluster_df["CDM_supertype_name"].str.split(" ").str[0]
cluster_df["CDM_class_id"]  = cluster_df["CDM_class_name"].str.split(" ").str[0]

#Reorder cluster_df
cluster_df=cluster_df.sort_values(['CDM_cluster_id', 'CDM_supertype_id', 'CDM_subclass_id', 'CDM_class_id'])


# Set thr on CDM_cluster_avg_correlation at the supertype level

## Set Thr by using a DoubleMAD for supertypes

### Compute DoubleMAD

In [231]:
# Calculate for each supertype_name_thr the DoubleMAD
DblMAD_supertype = metadata.groupby(['CDM_subclass_name', 'CDM_supertype_name']).agg(
                                        med=('CDM_cluster_avg_correlation', 'median'),
                                        DblMAD_low=('CDM_cluster_avg_correlation', lambda x: DoubleMAD(x)[0]),
                                        DblMAD_high=('CDM_cluster_avg_correlation', lambda x: DoubleMAD(x)[1])
                                    ).reset_index()
DblMAD_supertype = DblMAD_supertype[DblMAD_supertype['med'].notna()]


# Reset factor levels
DblMAD_supertype['CDM_subclass_name'] = DblMAD_supertype['CDM_subclass_name'].astype(str)
DblMAD_supertype['CDM_subclass_name'] = pd.Categorical(DblMAD_supertype['CDM_subclass_name'], categories = np.unique(cluster_df['CDM_subclass_name'][cluster_df['CDM_subclass_name'].isin(DblMAD_supertype['CDM_subclass_name'])]))

DblMAD_supertype['CDM_supertype_name'] = DblMAD_supertype['CDM_supertype_name'].astype(str)
DblMAD_supertype['CDM_supertype_name'] = pd.Categorical(DblMAD_supertype['CDM_supertype_name'], categories = np.unique(cluster_df['CDM_supertype_name'][cluster_df['CDM_supertype_name'].isin(DblMAD_supertype['CDM_supertype_name'])]))

# Set thr as median - 2*DoubleMAD_low
DblMAD_supertype['thr'] = DblMAD_supertype['med'] - 2*DblMAD_supertype['DblMAD_low']
DblMAD_supertype.loc[DblMAD_supertype['DblMAD_low']==0, 'thr'] = np.nan

#Create a list of DblMAD_supertype for each subclass
DblMAD_supertypeThr_subclasslist = [group for _, group in DblMAD_supertype.groupby('CDM_subclass_name')]


### Update metadata

In [232]:
####### Update meta_subclass_list #######################
metadata_subclass_list = [group for _, group in metadata.groupby('CDM_subclass_name')]


for i in range(len(metadata_subclass_list)):
    meta = metadata_subclass_list[i]
    DblMAD= DblMAD_supertypeThr_subclasslist[i]
    meta = meta.merge(DblMAD[['CDM_supertype_name','thr']], how='left', on='CDM_supertype_name')
    meta.rename(columns={'thr': 'CDM_supertype_thr'}, inplace=True)
    meta['CDM_cluster_thr_criteria'] = np.where(meta['CDM_cluster_avg_correlation'] >= meta['CDM_supertype_thr'], 'Passed', 'Failed')
    meta.loc[meta['CDM_supertype_thr'].isna(),'CDM_supertype_thr_criteria'] = 'Passed'
    metadata_subclass_list[i] = meta

## Update thr for clear bimodal distribution

### Update DoubleMAD Thr

#### Find supertype with bimodal distributrion

In [233]:
# Create a list of supertype dataframe in each subclass of metadata_subclass_list
meta_supertypeThr_subclasslist = []
for i in range(len(metadata_subclass_list)):
    meta= metadata_subclass_list[i]
    #meta['CDM_supertype_name'] = meta['CDM_supertype_name'].astype(str)
    #meta['CDM_supertype_name'] = pd.Categorical(meta['CDM_supertype_name'], categories=cluster_df['CDM_supertype_name'][cluster_df['CDM_supertype_name'].isin(meta['CDM_supertype_name'].unique())].unique())
    meta_supertype_list = [group for _, group in meta.groupby('CDM_supertype_name')]
    meta_supertypeThr_subclasslist.append(meta_supertype_list)

In [234]:
# Find which supertype has a bimodal distribution
bimodal_supertypeThr_subclasslist=[]
for i in range(len(meta_supertypeThr_subclasslist)):
    meta_subclass = meta_supertypeThr_subclasslist[i]
    bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['is.bimodal'](r_CDM_vector)
        bimodal_supertype_list.append(x[0])
    bimodal_supertypeThr_subclasslist.append(bimodal_supertype_list)

In [235]:
# Create meta_bimodal_supertypeThr_subclasslist
meta_bimodal_supertypeThr_subclasslist = []
for i in range(len(meta_supertypeThr_subclasslist)):
    meta_subclass = meta_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        if bimodal_supertypeThr_subclasslist[i][j] == True:
            meta_bimodal_supertype_list.append(meta)
    meta_bimodal_supertypeThr_subclasslist.append(meta_bimodal_supertype_list)
    
# Remove empty list
meta_bimodal_supertypeThr_subclasslist = [x for x in meta_bimodal_supertypeThr_subclasslist if x != []]

In [236]:
# Extract the column list CDM_supertype_name
bimodal_supertypeThr = []
for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        meta_bimodal_supertype_list.append(meta['CDM_supertype_name'].unique()[0])
    bimodal_supertypeThr.append(meta_bimodal_supertype_list)

bimodal_supertypeThr = [item for sublist in bimodal_supertypeThr for item in sublist]

#### Compute diffmode

In [237]:
# Calculate modes for each supertype with bimodal distribution
mode_bimodal_supertypeThr_subclasslist = []
for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    mode_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['Modes'](r_CDM_vector)
        x = np.sort(x[0])
        mode_bimodal_supertype_list.append(x)
    mode_bimodal_supertypeThr_subclasslist.append(mode_bimodal_supertype_list)

In [238]:
# Compute the difference between the two modes
diffmode_bimodal_supertypeThr_subclasslist = []
for i in range(len(mode_bimodal_supertypeThr_subclasslist)):
    meta_subclass = mode_bimodal_supertypeThr_subclasslist[i]
    diffmode_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        diffmode_bimodal_supertype_list.append(meta[1]-meta[0])
    diffmode_bimodal_supertypeThr_subclasslist.append(diffmode_bimodal_supertype_list)

In [239]:
# keep only supertype in meta_bimodal_supertypeThr_subclasslist if diffmode_bimodal_supertypeThr_subclasslist <0.05
meta_diffmode_bimodal_supertypeThr_subclasslist = []
for i in range(len(diffmode_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        if diffmode_bimodal_supertypeThr_subclasslist[i][j] < 0.05:
            meta_bimodal_supertype_list.append(meta)
    meta_diffmode_bimodal_supertypeThr_subclasslist.append(meta_bimodal_supertype_list)

# Remove empty list
meta_diffmode_bimodal_supertypeThr_subclasslist = [x for x in meta_diffmode_bimodal_supertypeThr_subclasslist if x != []]

In [240]:
# Extract the supertypes with diffmode <0.05
diffmode_bimodal_supertypeThr = []
for i in range(len(meta_diffmode_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_diffmode_bimodal_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        meta_bimodal_supertype_list.append(meta['CDM_supertype_name'].unique()[0])
    diffmode_bimodal_supertypeThr.append(meta_bimodal_supertype_list)

diffmode_bimodal_supertypeThr = [item for sublist in diffmode_bimodal_supertypeThr for item in sublist]

#### Compute local minimun

In [241]:
# Create list of prob dens for supertypes with bimodal distribution
prob_dens_bimodal_supertypeThr_subclasslist = []
for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    prob_dens_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['density'](r_CDM_vector)
        prob_dens_bimodal_supertype_list.append(x)
    prob_dens_bimodal_supertypeThr_subclasslist.append(prob_dens_bimodal_supertype_list)

In [242]:
# Find the local minimum between the two modes
lmin_bimodal_supertypeThr_subclasslist = []
for i in range(len(prob_dens_bimodal_supertypeThr_subclasslist)):
    dens_list = prob_dens_bimodal_supertypeThr_subclasslist[i]
    mode_list = mode_bimodal_supertypeThr_subclasslist[i]
    lmin_bimodal_supertype_list = []
    for j in range(len(dens_list)):
        dens = dens_list[j]
        mode = mode_list[j]
        # Find the local minimum between the two modes
        x = np.array(dens[0])
        y = np.array(dens[1])
        # Find the local minimum between the two modes
        lmin = x[(x >= mode[0]) & (x <= mode[1])][np.argmin(y[(x >= mode[0]) & (x <= mode[1])])]
        lmin_bimodal_supertype_list.append(lmin)
    lmin_bimodal_supertypeThr_subclasslist.append(lmin_bimodal_supertype_list)

In [243]:
# Calculate the median of each 'bimodal' supertype
med_bimodal_supertypeThr_subclasslist = []
for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    med_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        med_bimodal_supertype_list.append(meta['CDM_cluster_avg_correlation'].median())
    med_bimodal_supertypeThr_subclasslist.append(med_bimodal_supertype_list)

##### Which supertype has a local minimum greater than the median+0.05

In [244]:
# Which supertype has a local minimum greater than the median+0.05
lminsup005_bimodal_supertypeThr_subclasslist = []
for i in range(len(lmin_bimodal_supertypeThr_subclasslist)):
    lmin_list = lmin_bimodal_supertypeThr_subclasslist[i]
    med_list = med_bimodal_supertypeThr_subclasslist[i]
    lminsup005_bimodal_supertype_list = []
    for j in range(len(lmin_list)):
        lmin = lmin_list[j]
        med = med_list[j]
        if lmin > med+0.05:
            lminsup005_bimodal_supertype_list.append(True)
        else:
            lminsup005_bimodal_supertype_list.append(False)
    lminsup005_bimodal_supertypeThr_subclasslist.append(lminsup005_bimodal_supertype_list)


In [245]:
# keep only supertype in meta_bimodal_supertypeThr_subclasslist if lmin005_bimodal_supertypeThr_subclasslist is True
meta_lminsup005_bimodal_supertypeThr_subclasslist=[]
for i in range(len(lminsup005_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        if lminsup005_bimodal_supertypeThr_subclasslist[i][j] == True:
            meta_bimodal_supertype_list.append(meta)
    meta_lminsup005_bimodal_supertypeThr_subclasslist.append(meta_bimodal_supertype_list)

# Remove empty list
meta_lminsup005_bimodal_supertypeThr_subclasslist = [x for x in meta_lminsup005_bimodal_supertypeThr_subclasslist if x != []]

In [246]:
# Retrieve the name of the supertype with lmim between med-0.05 & med+0.05
lminsup005_bimodal_supertypeThr = []
for i in range(len(meta_lminsup005_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_lminsup005_bimodal_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        meta_bimodal_supertype_list.append(meta['CDM_supertype_name'].unique()[0])
    lminsup005_bimodal_supertypeThr.append(meta_bimodal_supertype_list)

lminsup005_bimodal_supertypeThr = [item for sublist in lminsup005_bimodal_supertypeThr for item in sublist]

##### Which supertype has a local minimum between med-0.05 and med+0.05

In [247]:
# Which supertype has a local minimum between med-0.05 and med+0.05
lmin005_bimodal_supertypeThr_subclasslist = []
for i in range(len(lmin_bimodal_supertypeThr_subclasslist)):
    lmin_list = lmin_bimodal_supertypeThr_subclasslist[i]
    med_list = med_bimodal_supertypeThr_subclasslist[i]
    lmin005_bimodal_supertype_list = []
    for j in range(len(lmin_list)):
        lmin = lmin_list[j]
        med = med_list[j]
        if (lmin > med-0.05) & (lmin < med+0.05):
            lmin005_bimodal_supertype_list.append(True)
        else:
            lmin005_bimodal_supertype_list.append(False)
    lmin005_bimodal_supertypeThr_subclasslist.append(lmin005_bimodal_supertype_list)


In [248]:
# keep only supertype in meta_bimodal_supertypeThr_subclasslist if lmin005_bimodal_supertypeThr_subclasslist is True
meta_lmin005_bimodal_supertypeThr_subclasslist=[]
for i in range(len(lmin005_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        if lmin005_bimodal_supertypeThr_subclasslist[i][j] == True:
            meta_bimodal_supertype_list.append(meta)
    meta_lmin005_bimodal_supertypeThr_subclasslist.append(meta_bimodal_supertype_list)

# Remove empty list
meta_lmin005_bimodal_supertypeThr_subclasslist = [x for x in meta_lmin005_bimodal_supertypeThr_subclasslist if x != []]

In [249]:
# Retrieve the name of the supertype with lmim between med-0.05 & med+0.05
lmin005_bimodal_supertypeThr = []
for i in range(len(meta_lmin005_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_lmin005_bimodal_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        meta_bimodal_supertype_list.append(meta['CDM_supertype_name'].unique()[0])
    lmin005_bimodal_supertypeThr.append(meta_bimodal_supertype_list)

lmin005_bimodal_supertypeThr = [item for sublist in lmin005_bimodal_supertypeThr for item in sublist]

In [250]:
lmin005_bimodal_supertypeThr = [item for item in lmin005_bimodal_supertypeThr if item not in diffmode_bimodal_supertypeThr]

#### Which supertypes to consider at the cluster level

In [251]:
supertype_name_1cl = cluster_df.groupby("CDM_supertype_name")["CDM_supertype_name"].count()
supertype_name_1cl = supertype_name_1cl[supertype_name_1cl == 1].index.tolist()

In [252]:
supertype_to_avgcluster = [item for item in lmin005_bimodal_supertypeThr if item not in supertype_name_1cl]

#### Supertypes to consider as bimodal

In [253]:
# supertypes predicted as bimodal distribution that need to be discarded
supertype_bimodal_med = [item for item in bimodal_supertypeThr if
    item in lmin005_bimodal_supertypeThr or
    item in lminsup005_bimodal_supertypeThr or
    item in diffmode_bimodal_supertypeThr
]

In [254]:
# Keep in meta_bimodal_supertypeThr_subclasslist only supertypes that are not in supertype_bimodal_med
for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    meta_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        if meta['CDM_supertype_name'].unique()[0] not in supertype_bimodal_med:
            meta_bimodal_supertype_list.append(meta)
    meta_bimodal_supertypeThr_subclasslist[i] = meta_bimodal_supertype_list
    
#Remove empty list
meta_bimodal_supertypeThr_subclasslist = [x for x in meta_bimodal_supertypeThr_subclasslist if x != []]

In [255]:
# Retrieve the name of the subclasses that are in meta_bimodal_supertypeThr_subclasslist
bimodal_supertypeThr_subclass = []
for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    bimodal_supertypeThr_subclass_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        bimodal_supertypeThr_subclass_list.append(meta['CDM_subclass_name'].unique()[0])
    bimodal_supertypeThr_subclass.append(bimodal_supertypeThr_subclass_list)

bimodal_supertypeThr_subclass = [item for sublist in bimodal_supertypeThr_subclass for item in sublist]
bimodal_supertypeThr_subclass = list(dict.fromkeys(bimodal_supertypeThr_subclass))

In [256]:
# Calculate modes for each supertype with bimodal distribution
mode_bimodal_supertypeThr_subclasslist = []
for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    mode_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['Modes'](r_CDM_vector)
        x = np.sort(x[0])
        mode_bimodal_supertype_list.append(x)
    mode_bimodal_supertypeThr_subclasslist.append(mode_bimodal_supertype_list)

In [257]:
# Create list of prob dens for supertypes with bimodal distribution
prob_dens_bimodal_supertypeThr_subclasslist = []
for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    prob_dens_bimodal_supertype_list = []
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['density'](r_CDM_vector)
        prob_dens_bimodal_supertype_list.append(x)
    prob_dens_bimodal_supertypeThr_subclasslist.append(prob_dens_bimodal_supertype_list)


In [258]:
# Find the local minimum between the two modes
lmin_bimodal_supertypeThr_subclasslist = []
for i in range(len(prob_dens_bimodal_supertypeThr_subclasslist)):
    dens_list = prob_dens_bimodal_supertypeThr_subclasslist[i]
    mode_list = mode_bimodal_supertypeThr_subclasslist[i]
    lmin_bimodal_supertype_list = []
    for j in range(len(dens_list)):
        dens = dens_list[j]
        mode = mode_list[j]
        # Find the local minimum between the two modes
        x = np.array(dens[0])
        y = np.array(dens[1])
        # Find the local minimum between the two modes
        lmin = x[(x >= mode[0]) & (x <= mode[1])][np.argmin(y[(x >= mode[0]) & (x <= mode[1])])]
        lmin_bimodal_supertype_list.append(lmin)
    lmin_bimodal_supertypeThr_subclasslist.append(lmin_bimodal_supertype_list)


In [259]:
lmin_bimodal_supertypeThr_subclasslist_df = []

for i in range(len(meta_bimodal_supertypeThr_subclasslist)):
    lmin_subclass = lmin_bimodal_supertypeThr_subclasslist[i]
    meta_subclass = meta_bimodal_supertypeThr_subclasslist[i]
    
    subclass_df_list = []  # Create a list to store DataFrames for this subclass
    
    for j in range(len(meta_subclass)):
        meta = meta_subclass[j]
        lmin = lmin_subclass[j]
        
        # Create a DataFrame with scalar values
        df = pd.DataFrame({'CDM_subclass_name': [meta['CDM_subclass_name'].unique()[0]],
                           'CDM_supertype_name': [meta['CDM_supertype_name'].unique()[0]],
                           'lmin': [lmin]})
        
        subclass_df_list.append(df)  # Append the DataFrame to the list
        
        # Concatenate DataFrames within the subclass_df_list
        subclass_df = pd.concat(subclass_df_list, ignore_index=True)
    lmin_bimodal_supertypeThr_subclasslist_df.append(subclass_df)

In [260]:
for i in range(len(DblMAD_supertypeThr_subclasslist)):
    DblMAD = DblMAD_supertypeThr_subclasslist[i]
    for j in range(len(lmin_bimodal_supertypeThr_subclasslist_df)):
        lmin = lmin_bimodal_supertypeThr_subclasslist_df[j]
        for k in range(DblMAD.shape[0]):
            for m in range(lmin.shape[0]):
                if DblMAD['CDM_subclass_name'].iloc[k] == lmin['CDM_subclass_name'].iloc[m] and DblMAD['CDM_supertype_name'].iloc[k] == lmin['CDM_supertype_name'].iloc[m]:
                    DblMAD['thr'].iloc[k] = lmin['lmin'].iloc[m]
    DblMAD_supertypeThr_subclasslist[i] = DblMAD

/scratch/fast/11930825/ipykernel_245685/3220364047.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DblMAD['thr'].iloc[k] = lmin['lmin'].iloc[m]


### Update data

In [261]:
####### Update metadata_subclass_list #######################
for i in range(len(metadata_subclass_list)):
    meta= metadata_subclass_list[i]
    DblMAD = DblMAD_supertypeThr_subclasslist[i]
    meta = meta.drop('CDM_supertype_thr', axis=1)
    meta = meta.drop('CDM_supertype_thr_criteria', axis=1)
    meta = meta.merge(DblMAD[['CDM_supertype_name','thr']], how='left', on='CDM_supertype_name')
    meta.rename(columns={'thr': 'CDM_supertype_thr'}, inplace=True)
    meta['CDM_supertype_thr_criteria'] = 'Passed' 
    meta.loc[meta['CDM_cluster_avg_correlation'] < meta['CDM_supertype_thr'], 'CDM_supertype_thr_criteria'] = 'Failed'
    meta.loc[meta['CDM_supertype_thr'].isna(),'CDM_supertype_thr_criteria'] = 'Passed'
    metadata_subclass_list[i] =  meta

In [262]:
DblMAD_supertype_adj = pd.concat(DblMAD_supertypeThr_subclasslist)
DblMAD_supertype_adj.rename(columns={DblMAD_supertype_adj.columns[5]: 'CDM_supertype_thr'}, inplace=True)
DblMAD_supertype_adj = DblMAD_supertype_adj[~DblMAD_supertype_adj.CDM_supertype_name.isin(supertype_to_avgcluster)]

In [263]:
########### update metadata #############
# Filter supertype_name_thr to exclude values in supertype_to_avgcluster
supertype_name_thr = cluster_df['CDM_supertype_name'].unique()
supertype_name_thr =  [supertype for supertype in supertype_name_thr if supertype not in supertype_to_avgcluster]

# Drop 'CDM_supertype_thr' and 'CDM_supertype_thr_criteria' columns
metadata.drop(columns=['CDM_supertype_thr'], inplace=True, errors='ignore')
metadata.drop(columns=['CDM_supertype_thr_criteria'], inplace=True, errors='ignore')


# Merge 'CDM_supertype_thr' column from DblMAD_supertype_adj
metadata = metadata.merge(DblMAD_supertype_adj[['CDM_supertype_name', 'CDM_supertype_thr']], on='CDM_supertype_name', how='left')

# Set 'CDM_supertype_thr_criteria' based on conditions
metadata.loc[metadata['CDM_cluster_avg_correlation'] < metadata['CDM_supertype_thr'], 'CDM_supertype_thr_criteria'] = 'Failed'
metadata.loc[metadata['CDM_supertype_thr'].isna() & metadata['CDM_supertype_name'].isin(supertype_name_thr),'CDM_supertype_thr_criteria'] = 'Passed'

## Recompute Thr after discarding low quality cells from the bimodal distribution

### Update DoubleMAD Thr

In [264]:
# subset the metadata_subclass_list to only include the metadata for the subclass(es) that encompass supertypes with bimodal distribution
meta_bimodal_subclass_list = [meta for meta in metadata_subclass_list if meta['CDM_subclass_name'].unique()[0] in bimodal_supertypeThr_subclass]

# subset the DblMAD_supertypeThr_subclasslist to only include the DblMAD for the subclass(es) that encompass supertypes with bimodal distribution
DblMAD_bimodal_supertypeThr_subclasslist = [DblMAD for DblMAD in DblMAD_supertypeThr_subclasslist if DblMAD['CDM_subclass_name'].unique()[0] in bimodal_supertypeThr_subclass]


In [265]:
bimodal_supertype_Thr= [item for item in bimodal_supertypeThr if item not in supertype_bimodal_med]

In [266]:
for i in range(len(DblMAD_bimodal_supertypeThr_subclasslist)):
    DblMAD = DblMAD_bimodal_supertypeThr_subclasslist[i]
    meta = meta_bimodal_subclass_list[i]
    meta = meta[(meta['CDM_supertype_thr_criteria']=='Passed') & meta['CDM_supertype_name'].isin(bimodal_supertype_Thr)]
    meta['CDM_supertype_name'] = meta['CDM_supertype_name'].astype(str)
    meta['CDM_supertype_name'] = pd.Categorical(meta['CDM_supertype_name'], categories=cluster_df['CDM_supertype_name'][cluster_df['CDM_supertype_name'].isin(meta['CDM_supertype_name'].unique())].unique())
    DblMAD_final = meta.groupby('CDM_supertype_name').agg(
                                        med=('CDM_cluster_avg_correlation', 'median'),
                                        DblMAD_low=('CDM_cluster_avg_correlation', lambda x: DoubleMAD(x)[0]),
                                        DblMAD_high=('CDM_cluster_avg_correlation', lambda x: DoubleMAD(x)[1])
                                    ).reset_index()
    DblMAD_final = DblMAD_final[DblMAD_final['med'].notna()]
    # Set thr
    DblMAD_final['thr'] = DblMAD_final['med'] - 2*DblMAD_final['DblMAD_low']
    for index_j, row_j in DblMAD.iterrows():
        for index_k, row_k in DblMAD_final.iterrows():
            if row_j['CDM_supertype_name'] == row_k['CDM_supertype_name']:
                DblMAD.loc[index_j,'thr'] = row_k['thr']
    DblMAD_bimodal_supertypeThr_subclasslist[i] = DblMAD

/scratch/fast/11930825/ipykernel_245685/3617998046.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta['CDM_supertype_name'] = meta['CDM_supertype_name'].astype(str)
/scratch/fast/11930825/ipykernel_245685/3617998046.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta['CDM_supertype_name'] = pd.Categorical(meta['CDM_supertype_name'], categories=cluster_df['CDM_supertype_name'][cluster_df['CDM_supertype_name'].isin(meta['CDM_supertype_name'].unique())].unique())
/scratch/fast/11930825/ipykernel_245

In [267]:
# Replace thr in DblMAD_supertypeThr_subclasslist with DblMAD_bimodal_supertypeThr_subclasslist
for i in range(len(DblMAD_supertypeThr_subclasslist)):
    DblMAD = DblMAD_supertypeThr_subclasslist[i]
    for j in range(len(DblMAD_bimodal_supertypeThr_subclasslist)):
        DblMAD_bimodal = DblMAD_bimodal_supertypeThr_subclasslist[j]
        for k in range(DblMAD.shape[0]):
            for m in range(DblMAD_bimodal.shape[0]):
                if DblMAD['CDM_supertype_name'].iloc[k] == DblMAD_bimodal['CDM_supertype_name'].iloc[m]:
                    DblMAD['thr'].iloc[k] = DblMAD_bimodal['thr'].iloc[m]
    DblMAD_supertypeThr_subclasslist[i] = DblMAD

/scratch/fast/11930825/ipykernel_245685/3769546502.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DblMAD['thr'].iloc[k] = DblMAD_bimodal['thr'].iloc[m]


### Update data

In [268]:
####### Update metadata_subclass_list #######################
for i in range(len(metadata_subclass_list)):
    meta= metadata_subclass_list[i]
    DblMAD = DblMAD_supertypeThr_subclasslist[i]
    meta = meta.drop('CDM_supertype_thr', axis=1)
    meta = meta.drop('CDM_supertype_thr_criteria', axis=1)
    meta = meta.merge(DblMAD[['CDM_supertype_name','thr']], how='left', on='CDM_supertype_name')
    meta.rename(columns={'thr': 'CDM_supertype_thr'}, inplace=True)
    meta['CDM_supertype_thr_criteria'] = np.where(meta['CDM_cluster_avg_correlation'] >= meta['CDM_supertype_thr'], 'Passed', 'Failed')
    meta.loc[(meta['CDM_supertype_thr'].isna()) & meta['CDM_supertype_name'].isin(supertype_name_thr) ,'CDM_supertype_thr_criteria'] = 'Passed'
    metadata_subclass_list[i] =  meta

In [269]:
# Merge DblMAD_supertypeThr_subclasslist
DblMAD_supertype_final = pd.concat(DblMAD_supertypeThr_subclasslist)
DblMAD_supertype_final.rename(columns={DblMAD_supertype_final.columns[5]: "CDM_supertype_thr"}, inplace=True) 

In [270]:
########### update metadata #############

# Drop 'CDM_supertype_thr' and 'CDM_supertype_thr_criteria' columns
metadata.drop(columns=['CDM_supertype_thr'], inplace=True, errors='ignore')
metadata.drop(columns=['CDM_supertype_thr_criteria'], inplace=True, errors='ignore')

# Merge 'CDM_supertype_thr' column from DblMAD_supertype_adj
metadata = metadata.merge(DblMAD_supertype_final[['CDM_supertype_name', 'CDM_supertype_thr']], on='CDM_supertype_name', how='left')

# Set 'CDM_supertype_thr_criteria' based on conditions
metadata['CDM_supertype_thr_criteria'] = np.where(metadata['CDM_cluster_avg_correlation'] >= metadata['CDM_supertype_thr'], 'Passed', 'Failed')
metadata.loc[(metadata['CDM_supertype_thr'].isna()) & metadata['CDM_supertype_name'].isin(supertype_name_thr),'CDM_supertype_thr_criteria'] = 'Passed'

# Set Thr on CDM_cluster_avg_correlation at the cluster level

## Set Thr by using a DoubleMAD for clusters

In [271]:
cluster_name_thr = cluster_df.loc[cluster_df['CDM_supertype_name'].isin(supertype_to_avgcluster), 'CDM_cluster_name'].tolist()

### Compute DoubleMAD

In [272]:
# Calculate for each cluster_name_thr the DoubleMAD
DblMAD_cluster = metadata[metadata['CDM_cluster_name'].isin(cluster_name_thr)].groupby(['CDM_supertype_name', 'CDM_cluster_name']).agg(
                                        med=('CDM_cluster_avg_correlation', 'median'),
                                        DblMAD_low=('CDM_cluster_avg_correlation', lambda x: DoubleMAD(x)[0]),
                                        DblMAD_high=('CDM_cluster_avg_correlation', lambda x: DoubleMAD(x)[1])
                                    ).reset_index()
DblMAD_cluster = DblMAD_cluster[DblMAD_cluster['med'].notna()]


# Reset factor levels
DblMAD_cluster['CDM_supertype_name'] = DblMAD_cluster['CDM_supertype_name'].astype(str)
DblMAD_cluster['CDM_supertype_name'] = pd.Categorical(DblMAD_cluster['CDM_supertype_name'], categories = np.unique(cluster_df['CDM_supertype_name'][cluster_df['CDM_supertype_name'].isin(DblMAD_cluster['CDM_supertype_name'])]))

DblMAD_cluster['CDM_cluster_name'] = DblMAD_cluster['CDM_cluster_name'].astype(str)
DblMAD_cluster['CDM_cluster_name'] = pd.Categorical(DblMAD_cluster['CDM_cluster_name'], categories = np.unique(cluster_df['CDM_cluster_name'][cluster_df['CDM_cluster_name'].isin(DblMAD_cluster['CDM_cluster_name'])]))

# Set thr as median - 2*DoubleMAD_low
DblMAD_cluster['thr'] = DblMAD_cluster['med'] - 2*DblMAD_cluster['DblMAD_low']
DblMAD_cluster.loc[DblMAD_cluster['DblMAD_low']==0, 'thr'] = np.nan

#Create a list of DblMAD_cluster for each cluster
DblMAD_clusterThr_supertypelist = [group for _, group in DblMAD_cluster.groupby('CDM_supertype_name')]



### Update metadata list

In [273]:
####### Update meta_supertype_list #######################
metadata_supertype_list = [group for _, group in metadata[metadata['CDM_cluster_name'].isin(cluster_name_thr)].groupby('CDM_supertype_name')]


for i in range(len(metadata_supertype_list)):
    meta= metadata_supertype_list[i]
    DblMAD = DblMAD_clusterThr_supertypelist[i]
    meta = meta.merge(DblMAD[['CDM_cluster_name','thr']], how='left', on='CDM_cluster_name')
    meta.rename(columns={'thr': 'CDM_cluster_thr'}, inplace=True)
    meta['CDM_cluster_thr_criteria'] = np.where(meta['CDM_cluster_avg_correlation'] >= meta['CDM_cluster_thr'], 'Passed', 'Failed')
    meta.loc[(meta['CDM_cluster_thr'].isna()) & meta['CDM_cluster_name'].isin(cluster_name_thr) ,'CDM_cluster_thr_criteria'] = 'Passed'
    metadata_supertype_list[i] = meta

In [274]:
########### update metadata #############

# Rename last column of DblMAD_cluster CDM_cluster_thr
DblMAD_cluster = DblMAD_cluster.rename(columns={DblMAD_cluster.columns[5]: "CDM_cluster_thr"})

# Drop 'CDM_supertype_thr' and 'CDM_supertype_thr_criteria' columns
metadata.drop(columns=['CDM_cluster_thr'], inplace=True, errors='ignore')
metadata.drop(columns=['CDM_cluster_thr_criteria'], inplace=True, errors='ignore')

# Merge 'CDM_cluster_thr' column from DblMAD_cluster
metadata = metadata.merge(DblMAD_cluster[['CDM_cluster_name', 'CDM_cluster_thr']], on='CDM_cluster_name', how='left')

# Set 'CDM_cluster_thr_criteria' based on conditions
metadata['CDM_cluster_thr_criteria'] = np.where(metadata['CDM_cluster_avg_correlation'] >= metadata['CDM_cluster_thr'], 'Passed', 'Failed')
metadata.loc[(metadata['CDM_cluster_thr'].isna()) & metadata['CDM_cluster_name'].isin(cluster_name_thr),'CDM_cluster_thr_criteria'] = 'Passed'

## Update thr for clear bimodal distribution

### Update DoubleMAD Thr

In [275]:
# Create a list of cluster dataframe in each elemet of metadata_supertype_list
meta_clusterThr_supertypelist = []
for i in range(len(metadata_supertype_list)):
    meta= metadata_supertype_list[i]
    meta_cluster_list = [group for _, group in meta.groupby('CDM_cluster_name')]
    meta_clusterThr_supertypelist.append(meta_cluster_list)

#### Find cluster with bimodal distribution

In [276]:
# Find which cluster has a bimodal distribution
bimodal_clusterThr_supertypelist=[]
for i in range(len(meta_clusterThr_supertypelist)):
    meta_supertype = meta_clusterThr_supertypelist[i]
    bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['is.bimodal'](r_CDM_vector)
        bimodal_cluster_list.append(x[0])
    bimodal_clusterThr_supertypelist.append(bimodal_cluster_list)

In [277]:
# Create meta_bimodal_clusterThr_supertypelist
meta_bimodal_clusterThr_supertypelist = []
for i in range(len(meta_clusterThr_supertypelist)):
    meta_supertype = meta_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        if bimodal_clusterThr_supertypelist[i][j] == True:
            meta_bimodal_cluster_list.append(meta)
    meta_bimodal_clusterThr_supertypelist.append(meta_bimodal_cluster_list)
    
# Remove empty list
meta_bimodal_clusterThr_supertypelist = [x for x in meta_bimodal_clusterThr_supertypelist if x != []]


In [278]:
 # Extract the column list CDM_cluster_name
bimodal_clusterThr = []
for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        meta_bimodal_cluster_list.append(meta['CDM_cluster_name'].unique()[0])
    bimodal_clusterThr.append(meta_bimodal_cluster_list)

bimodal_clusterThr = [item for sublist in bimodal_clusterThr for item in sublist]

#### Compute diffmode

In [279]:
# Calculate modes for each cluster with bimodal distribution
mode_bimodal_clusterThr_supertypelist = []
for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    mode_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['Modes'](r_CDM_vector)
        x = np.sort(x[0])
        mode_bimodal_cluster_list.append(x)
    mode_bimodal_clusterThr_supertypelist.append(mode_bimodal_cluster_list)

In [280]:
# Compute the difference between the two modes
diffmode_bimodal_clusterThr_supertypelist = []
for i in range(len(mode_bimodal_clusterThr_supertypelist)):
    meta_supertype = mode_bimodal_clusterThr_supertypelist[i]
    diffmode_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        diffmode_bimodal_cluster_list.append(meta[1]-meta[0])
    diffmode_bimodal_clusterThr_supertypelist.append(diffmode_bimodal_cluster_list)

In [281]:
# keep only cluster in meta_bimodal_clusterThr_supertypelist if diffmode_bimodal_clusterThr_supertypelist <0.05
meta_diffmode_bimodal_clusterThr_supertypelist = []
for i in range(len(diffmode_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        if diffmode_bimodal_clusterThr_supertypelist[i][j] < 0.05:
            meta_bimodal_cluster_list.append(meta)
    meta_diffmode_bimodal_clusterThr_supertypelist.append(meta_bimodal_cluster_list)

# Remove empty list
meta_diffmode_bimodal_clusterThr_supertypelist = [x for x in meta_diffmode_bimodal_clusterThr_supertypelist if x != []]


In [282]:
# Do not consider as bimodal if diffmode < 0.05
diffmode_bimodal_clusterThr = []
for i in range(len(meta_diffmode_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_diffmode_bimodal_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        meta_bimodal_cluster_list.append(meta['CDM_cluster_name'].unique()[0])
    diffmode_bimodal_clusterThr.append(meta_bimodal_cluster_list)

diffmode_bimodal_clusterThr = [item for sublist in diffmode_bimodal_clusterThr for item in sublist]


#### Compute local minimum

In [283]:
# Create list of prob dens for clusters with bimodal distribution
prob_dens_bimodal_clusterThr_supertypelist = []
for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    prob_dens_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['density'](r_CDM_vector)
        prob_dens_bimodal_cluster_list.append(x)
    prob_dens_bimodal_clusterThr_supertypelist.append(prob_dens_bimodal_cluster_list)

In [284]:
# Find the local minimum between the two modes
lmin_bimodal_clusterThr_supertypelist = []
for i in range(len(prob_dens_bimodal_clusterThr_supertypelist)):
    dens_list = prob_dens_bimodal_clusterThr_supertypelist[i]
    mode_list = mode_bimodal_clusterThr_supertypelist[i]
    lmin_bimodal_cluster_list = []
    for j in range(len(dens_list)):
        dens = dens_list[j]
        mode = mode_list[j]
        # Find the local minimum between the two modes
        x = np.array(dens[0])
        y = np.array(dens[1])
        # Find the local minimum between the two modes
        lmin = x[(x >= mode[0]) & (x <= mode[1])][np.argmin(y[(x >= mode[0]) & (x <= mode[1])])]
        lmin_bimodal_cluster_list.append(lmin)
    lmin_bimodal_clusterThr_supertypelist.append(lmin_bimodal_cluster_list)

In [285]:
# Calculate the median of each 'bimodal' cluster
med_bimodal_clusterThr_supertypelist = []
for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    med_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        med_bimodal_cluster_list.append(meta['CDM_cluster_avg_correlation'].median())
    med_bimodal_clusterThr_supertypelist.append(med_bimodal_cluster_list)

##### Which cluster has a local minimum greater than the median+0.05

In [286]:
# Which cluster has a local minimum greater than the median+0.05
lminsup005_bimodal_clusterThr_supertypelist = []
for i in range(len(lmin_bimodal_clusterThr_supertypelist)):
    lmin_list = lmin_bimodal_clusterThr_supertypelist[i]
    med_list = med_bimodal_clusterThr_supertypelist[i]
    lminsup005_bimodal_cluster_list = []
    for j in range(len(lmin_list)):
        lmin = lmin_list[j]
        med = med_list[j]
        if lmin > med+0.05:
            lminsup005_bimodal_cluster_list.append(True)
        else:
            lminsup005_bimodal_cluster_list.append(False)
    lminsup005_bimodal_clusterThr_supertypelist.append(lminsup005_bimodal_cluster_list)

In [287]:
# keep only cluster in meta_bimodal_clusterThr_supertypelist if lmin005_bimodal_clusterThr_supertypelist is True
meta_lminsup005_bimodal_clusterThr_supertypelist=[]
for i in range(len(lminsup005_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        if lminsup005_bimodal_clusterThr_supertypelist[i][j] == True:
            meta_bimodal_cluster_list.append(meta)
    meta_lminsup005_bimodal_clusterThr_supertypelist.append(meta_bimodal_cluster_list)

# Remove empty list
meta_lminsup005_bimodal_clusterThr_supertypelist = [x for x in meta_lminsup005_bimodal_clusterThr_supertypelist if x != []]

In [288]:
# Retrieve the name of the cluster with lmim greater than med+0.05
lminsup005_bimodal_clusterThr = []
for i in range(len(meta_lminsup005_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_lminsup005_bimodal_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        meta_bimodal_cluster_list.append(meta['CDM_cluster_name'].unique()[0])
    lminsup005_bimodal_clusterThr.append(meta_bimodal_cluster_list)

In [289]:
lminsup005_bimodal_clusterThr = [item for sublist in lminsup005_bimodal_clusterThr for item in sublist]

##### Which cluster has a local minimum between med-0.05 and med+0.05

In [290]:
# Which cluster has a local minimum between med-0.05 and med+0.05
lmin005_bimodal_clusterThr_supertypelist = []
for i in range(len(lmin_bimodal_clusterThr_supertypelist)):
    lmin_list = lmin_bimodal_clusterThr_supertypelist[i]
    med_list = med_bimodal_clusterThr_supertypelist[i]
    lmin005_bimodal_cluster_list = []
    for j in range(len(lmin_list)):
        lmin = lmin_list[j]
        med = med_list[j]
        if (lmin > med-0.05) & (lmin < med+0.05):
            lmin005_bimodal_cluster_list.append(True)
        else:
            lmin005_bimodal_cluster_list.append(False)
    lmin005_bimodal_clusterThr_supertypelist.append(lmin005_bimodal_cluster_list)

In [291]:
# keep only cluster in meta_bimodal_clusterThr_supertypelist if lmin005_bimodal_clusterThr_supertypelist is True
meta_lmin005_bimodal_clusterThr_supertypelist=[]
for i in range(len(lmin005_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        if lmin005_bimodal_clusterThr_supertypelist[i][j] == True:
            meta_bimodal_cluster_list.append(meta)
    meta_lmin005_bimodal_clusterThr_supertypelist.append(meta_bimodal_cluster_list)

# Remove empty list
meta_lmin005_bimodal_clusterThr_supertypelist = [x for x in meta_lmin005_bimodal_clusterThr_supertypelist if x != []]

In [292]:
# Retrieve the name of the cluster with lmim between med-0.05 & med+0.05
lmin005_bimodal_clusterThr = []
for i in range(len(meta_lmin005_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_lmin005_bimodal_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        meta_bimodal_cluster_list.append(meta['CDM_cluster_name'].unique()[0])
    lmin005_bimodal_clusterThr.append(meta_bimodal_cluster_list)

In [293]:
lmin005_bimodal_clusterThr = [item for sublist in lmin005_bimodal_clusterThr for item in sublist]
lmin005_bimodal_clusterThr = [item for item in lmin005_bimodal_clusterThr if item not in diffmode_bimodal_clusterThr]

#### Clusters to consider as bimodal

In [294]:
# clusters predicted as bimodal distribution that need to be discarded
cluster_bimodal_med = [item for item in bimodal_clusterThr if
    item in lmin005_bimodal_clusterThr or
    item in lminsup005_bimodal_clusterThr or
    item in diffmode_bimodal_clusterThr
]

In [295]:
# Keep in meta_bimodal_clusterThr_supertypelist only clusters that are not in cluster_bimodal_med
for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    meta_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        if meta['CDM_cluster_name'].unique()[0] not in cluster_bimodal_med:
            meta_bimodal_cluster_list.append(meta)
    meta_bimodal_clusterThr_supertypelist[i] = meta_bimodal_cluster_list
    
#Remove empty list
meta_bimodal_clusterThr_supertypelist = [x for x in meta_bimodal_clusterThr_supertypelist if x != []]


In [296]:
# Retrieve the name of the supertypees that are in meta_bimodal_clusterThr_supertypelist
bimodal_clusterThr_supertype = []
for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    bimodal_clusterThr_supertype_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        bimodal_clusterThr_supertype_list.append(meta['CDM_supertype_name'].unique()[0])
    bimodal_clusterThr_supertype.append(bimodal_clusterThr_supertype_list)

bimodal_clusterThr_supertype = [item for sublist in bimodal_clusterThr_supertype for item in sublist]
bimodal_clusterThr_supertype = list(dict.fromkeys(bimodal_clusterThr_supertype))

In [297]:
# Calculate modes for each cluster with bimodal distribution
mode_bimodal_clusterThr_supertypelist = []
for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    mode_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['Modes'](r_CDM_vector)
        x = np.sort(x[0])
        mode_bimodal_cluster_list.append(x)
    mode_bimodal_clusterThr_supertypelist.append(mode_bimodal_cluster_list)

In [298]:
# Create list of prob dens for clusters with bimodal distribution
prob_dens_bimodal_clusterThr_supertypelist = []
for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    prob_dens_bimodal_cluster_list = []
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        CDM_vector=meta['CDM_cluster_avg_correlation']
        r_CDM_vector = pandas2ri.py2rpy(CDM_vector)
        x= robjects.r['density'](r_CDM_vector)
        prob_dens_bimodal_cluster_list.append(x)
    prob_dens_bimodal_clusterThr_supertypelist.append(prob_dens_bimodal_cluster_list)

In [299]:
# Find the local minimum between the two modes
lmin_bimodal_clusterThr_supertypelist = []
for i in range(len(prob_dens_bimodal_clusterThr_supertypelist)):
    dens_list = prob_dens_bimodal_clusterThr_supertypelist[i]
    mode_list = mode_bimodal_clusterThr_supertypelist[i]
    lmin_bimodal_cluster_list = []
    for j in range(len(dens_list)):
        dens = dens_list[j]
        mode = mode_list[j]
        # Find the local minimum between the two modes
        x = np.array(dens[0])
        y = np.array(dens[1])
        # Find the local minimum between the two modes
        lmin = x[(x >= mode[0]) & (x <= mode[1])][np.argmin(y[(x >= mode[0]) & (x <= mode[1])])]
        lmin_bimodal_cluster_list.append(lmin)
    lmin_bimodal_clusterThr_supertypelist.append(lmin_bimodal_cluster_list)

In [300]:
lmin_bimodal_clusterThr_supertypelist_df = []

for i in range(len(meta_bimodal_clusterThr_supertypelist)):
    lmin_cluster = lmin_bimodal_clusterThr_supertypelist[i]
    meta_supertype = meta_bimodal_clusterThr_supertypelist[i]
    
    cluster_list = []  # Create a list to store DataFrames for this cluster
    
    for j in range(len(meta_supertype)):
        meta = meta_supertype[j]
        lmin = lmin_cluster[j]
        
        # Create a DataFrame with scalar values
        df = pd.DataFrame({'CDM_cluster_name': [meta['CDM_cluster_name'].unique()[0]],
                           'CDM_cluster_name': [meta['CDM_cluster_name'].unique()[0]],
                           'lmin': [lmin]})
        
        cluster_list.append(df)  # Append the DataFrame to the list
        
        # Concatenate DataFrames within the cluster_list
        cluster = pd.concat(cluster_list, ignore_index=True)
    lmin_bimodal_clusterThr_supertypelist_df.append(cluster)

In [301]:
#### Update the thr for in DblMAD_clusterThr_supertypelist in each cluster with bimodal distribution
for i in range(len(DblMAD_clusterThr_supertypelist)):
    DblMAD = DblMAD_clusterThr_supertypelist[i]
    for j in range(len(lmin_bimodal_clusterThr_supertypelist_df)):
        lmin = lmin_bimodal_clusterThr_supertypelist_df[j]
        for k in range(DblMAD.shape[0]):
            for m in range(lmin.shape[0]):
                if DblMAD['CDM_cluster_name'].iloc[k] == lmin['CDM_cluster_name'].iloc[m] and DblMAD['CDM_cluster_name'].iloc[k] == lmin['CDM_cluster_name'].iloc[m]:
                    DblMAD['thr'].iloc[k] = lmin['lmin'].iloc[m]
    DblMAD_clusterThr_supertypelist[i] = DblMAD

/scratch/fast/11930825/ipykernel_245685/4133917256.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DblMAD['thr'].iloc[k] = lmin['lmin'].iloc[m]


### Update data

In [302]:
####### Update metadata_supertype_list #######################
for i in range(len(metadata_supertype_list)):
    meta= metadata_supertype_list[i]
    DblMAD = DblMAD_clusterThr_supertypelist[i]
    meta = meta.drop('CDM_cluster_thr', axis=1, errors='ignore')
    meta = meta.drop('CDM_cluster_thr_criteria', axis=1, errors='ignore')
    meta = meta.merge(DblMAD[['CDM_cluster_name','thr']], how='left', on='CDM_cluster_name')
    meta.rename(columns={'thr': 'CDM_cluster_thr'}, inplace=True)
    meta['CDM_cluster_thr_criteria'] = np.where(meta['CDM_cluster_avg_correlation'] >= meta['CDM_cluster_thr'], 'Passed', 'Failed')
    meta.loc[(meta['CDM_cluster_thr'].isna()) & meta['CDM_cluster_name'].isin(cluster_name_thr) ,'CDM_cluster_thr_criteria'] = 'Passed'
    metadata_supertype_list[i] =  meta

In [303]:
DblMAD_cluster_adj = pd.concat(DblMAD_clusterThr_supertypelist)
DblMAD_cluster_adj.rename(columns={DblMAD_cluster_adj.columns[5]: 'CDM_cluster_thr'}, inplace=True)

In [304]:
########### update metadata #############

# Drop 'CDM_cluster_thr' and 'CDM_cluster_thr_criteria' columns
metadata.drop(columns=['CDM_cluster_thr'], inplace=True, errors='ignore')
metadata.drop(columns=['CDM_cluster_thr_criteria'], inplace=True, errors='ignore')


# Merge 'CDM_cluster_thr' column from DblMAD_cluster_adj
metadata = metadata.merge(DblMAD_cluster_adj[['CDM_cluster_name', 'CDM_cluster_thr']], on='CDM_cluster_name', how='left')

# Set 'CDM_cluster_thr_criteria' based on conditions
metadata.loc[metadata['CDM_cluster_avg_correlation'] < metadata['CDM_cluster_thr'], 'CDM_cluster_thr_criteria'] = 'Failed'
metadata.loc[metadata['CDM_cluster_thr'].isna() & metadata['CDM_cluster_name'].isin(cluster_name_thr),'CDM_cluster_thr_criteria'] = 'Passed'


## Recompute Thr after discarding low quality cells from the bimodal distribution

### Update DoubleMAD Thr

In [305]:
# subset the metadata_supertype_list to only include the metadata for the supertype(s) that encompass clusters with bimodal distribution
meta_bimodal_supertype_list = [meta for meta in metadata_supertype_list if meta['CDM_supertype_name'].unique()[0] in bimodal_clusterThr_supertype]

# subset the DblMAD_clusterThr_supertypelist to only include the DblMAD for the supertype(s) that encompass clusters with bimodal distribution
DblMAD_bimodal_clusterThr_supertypelist = [DblMAD for DblMAD in DblMAD_clusterThr_supertypelist if DblMAD['CDM_supertype_name'].unique()[0] in bimodal_clusterThr_supertype]

In [306]:
### Update DoubleMAD Thr
bimodal_cluster_Thr= [item for item in bimodal_clusterThr if item not in cluster_bimodal_med]

In [307]:
for i in range(len(DblMAD_bimodal_clusterThr_supertypelist)):
    DblMAD = DblMAD_bimodal_clusterThr_supertypelist[i]
    meta = meta_bimodal_supertype_list[i]
    meta = meta[(meta['CDM_cluster_thr_criteria']=='Passed') & meta['CDM_cluster_name'].isin(bimodal_cluster_Thr)]
    meta['CDM_cluster_name'] = meta['CDM_cluster_name'].astype(str)
    meta['CDM_cluster_name'] = pd.Categorical(meta['CDM_cluster_name'], categories=cluster_df['CDM_cluster_name'][cluster_df['CDM_cluster_name'].isin(meta['CDM_cluster_name'].unique())].unique())
    DblMAD_final = meta.groupby('CDM_cluster_name').agg(
                                        med=('CDM_cluster_avg_correlation', 'median'),
                                        DblMAD_low=('CDM_cluster_avg_correlation', lambda x: DoubleMAD(x)[0]),
                                        DblMAD_high=('CDM_cluster_avg_correlation', lambda x: DoubleMAD(x)[1])
                                    ).reset_index()
    DblMAD_final = DblMAD_final[DblMAD_final['med'].notna()]
    # Set thr
    DblMAD_final['thr'] = DblMAD_final['med'] - 2*DblMAD_final['DblMAD_low']
    for j in range(DblMAD.shape[0]):
        for k in range(DblMAD_final.shape[0]):
            if DblMAD['CDM_cluster_name'].iloc[j] == DblMAD_final['CDM_cluster_name'].iloc[k]:
                DblMAD['thr'].iloc[j] = DblMAD_final['thr'].iloc[k]
    DblMAD_bimodal_clusterThr_supertypelist[i] = DblMAD

/scratch/fast/11930825/ipykernel_245685/2975664826.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta['CDM_cluster_name'] = meta['CDM_cluster_name'].astype(str)
/scratch/fast/11930825/ipykernel_245685/2975664826.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta['CDM_cluster_name'] = pd.Categorical(meta['CDM_cluster_name'], categories=cluster_df['CDM_cluster_name'][cluster_df['CDM_cluster_name'].isin(meta['CDM_cluster_name'].unique())].unique())
/scratch/fast/11930825/ipykernel_245685/2975664826

### Update data

In [308]:
####### Update metadata_supertype_list #######################
for i in range(len(metadata_supertype_list)):
    meta= metadata_supertype_list[i]
    DblMAD = DblMAD_clusterThr_supertypelist[i]
    meta = meta.drop('CDM_cluster_thr', axis=1)
    meta = meta.drop('CDM_cluster_thr_criteria', axis=1)
    meta = meta.merge(DblMAD[['CDM_cluster_name','thr']], how='left', on='CDM_cluster_name')
    meta.rename(columns={'thr': 'CDM_cluster_thr'}, inplace=True)
    meta['CDM_cluster_thr_criteria'] = np.where(meta['CDM_cluster_avg_correlation'] >= meta['CDM_cluster_thr'], 'Passed', 'Failed')
    meta.loc[(meta['CDM_cluster_thr'].isna()) & meta['CDM_cluster_name'].isin(cluster_name_thr) ,'CDM_cluster_thr_criteria'] = 'Passed'
    metadata_supertype_list[i] =  meta

In [309]:
# Merge DblMAD_clusterThr_supertypelist
DblMAD_cluster_final = pd.concat(DblMAD_clusterThr_supertypelist)
DblMAD_cluster_final.rename(columns={DblMAD_cluster_final.columns[5]: "CDM_cluster_thr"}, inplace=True) 

In [310]:
########### update metadata #############

# Drop 'CDM_cluster_thr' and 'CDM_cluster_thr_criteria' columns
metadata.drop(columns=['CDM_cluster_thr'], inplace=True, errors='ignore')
metadata.drop(columns=['CDM_cluster_thr_criteria'], inplace=True, errors='ignore')

# Merge 'CDM_cluster_thr' column from DblMAD_cluster_adj
metadata = metadata.merge(DblMAD_cluster_final[['CDM_cluster_name', 'CDM_cluster_thr']], on='CDM_cluster_name', how='left')

# Set 'CDM_cluster_thr_criteria' based on conditions
metadata['CDM_cluster_thr_criteria'] = np.where(metadata['CDM_cluster_avg_correlation'] >= metadata['CDM_cluster_thr'], 'Passed', 'Failed')
metadata.loc[(metadata['CDM_cluster_thr'].isna()) & metadata['CDM_cluster_name'].isin(cluster_name_thr),'CDM_cluster_thr_criteria'] = 'Passed'


# Combine CDM_thr

## Update data

In [311]:
### Update metadata ###
metadata.drop(columns=['CDM_thr'], inplace=True, errors='ignore')
metadata.loc[metadata['CDM_supertype_name'].isin(supertype_name_thr),'CDM_thr'] = metadata.loc[metadata['CDM_supertype_name'].isin(supertype_name_thr),'CDM_supertype_thr']
metadata.loc[metadata['CDM_cluster_name'].isin(cluster_name_thr),'CDM_thr'] = metadata.loc[metadata['CDM_cluster_name'].isin(cluster_name_thr),'CDM_cluster_thr']

metadata.drop(columns=['CDM_thr_criteria'], inplace=True, errors='ignore')
metadata.loc[metadata['CDM_supertype_name'].isin(supertype_name_thr),'CDM_thr_criteria'] = metadata.loc[metadata['CDM_supertype_name'].isin(supertype_name_thr),'CDM_supertype_thr_criteria']
metadata.loc[metadata['CDM_cluster_name'].isin(cluster_name_thr),'CDM_thr_criteria'] = metadata.loc[metadata['CDM_cluster_name'].isin(cluster_name_thr),'CDM_cluster_thr_criteria']

In [312]:
metadata['CDM_thr_criteria'].value_counts()

Passed    4749365
Failed     612273
Name: CDM_thr_criteria, dtype: int64

# Save Data

## Load adata

In [818]:
#Load adata object
adata= sc.read_h5ad(os.path.join(input_dir, 'mouse_'+mouse+'_combined_fullpreprocessed_CDMv2.h5ad'))
adata

/allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/remimathieu/miniconda3/envs/rpy2-env/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.


AnnData object with n_obs × n_vars = 5361638 × 550
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x', 'max_y', 'barcodeCount', 'corrected_x', 'corrected_y', 'origin', 'rotation', 'section', 'n_genes_by_counts', 'total_counts', 'total_counts_Blank', 'pct_counts_Blank', 'total_counts_per_cell_volume', 'total_counts_Blank_per_cell_volume', 'n_genes', 'blank_qc', 'doublets', 'Solo_prediction', 'leiden', 'DoubletFinder_prediction', 'CDM_class_label', 'CDM_class_name', 'CDM_class_bootstrapping_probability', 'CDM_class_avg_correlation', 'CDM_subclass_label', 'CDM_subclass_name', 'CDM_subclass_bootstrapping_probability', 'CDM_subclass_avg_correlation', 'CDM_supertype_label', 'CDM_supertype_name', 'CDM_supertype_bootstrapping_probability', 'CDM_supertype_avg_correlation', 'CDM_cluster_label', 'CDM_cluster_name', 'CDM_cluster_alias', 'CDM_cluster_bootstrapping_probability', 'CDM_cluster_avg_correlation', 'min_corrected_x', 'min_corrected_y', 'spatial_x', 'spatial_y', 'o

## Add results in adata.obs

In [820]:
adata.obs.index = adata.obs['cell_label'].astype(str)

# order metadata in the same order as adata.obs
metadata.index = metadata['cell_label'].astype(str) 
metadata = metadata.loc[adata.obs.index]

# Replace adata.obs by metadata
adata.obs=metadata
adata

AnnData object with n_obs × n_vars = 5361638 × 550
    obs: 'fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x', 'max_y', 'barcodeCount', 'corrected_x', 'corrected_y', 'origin', 'rotation', 'section', 'n_genes_by_counts', 'total_counts', 'total_counts_Blank', 'pct_counts_Blank', 'total_counts_per_cell_volume', 'total_counts_Blank_per_cell_volume', 'n_genes', 'blank_qc', 'doublets', 'Solo_prediction', 'leiden', 'DoubletFinder_prediction', 'CDM_class_label', 'CDM_class_name', 'CDM_class_bootstrapping_probability', 'CDM_class_avg_correlation', 'CDM_subclass_label', 'CDM_subclass_name', 'CDM_subclass_bootstrapping_probability', 'CDM_subclass_avg_correlation', 'CDM_supertype_label', 'CDM_supertype_name', 'CDM_supertype_bootstrapping_probability', 'CDM_supertype_avg_correlation', 'CDM_cluster_label', 'CDM_cluster_name', 'CDM_cluster_alias', 'CDM_cluster_bootstrapping_probability', 'CDM_cluster_avg_correlation', 'CDM_class_color', 'CDM_subclass_color', 'CDM_supertype_color', 'C

In [821]:
# Create directory ThrCDM
os.makedirs(os.path.join(input_dir, 'ThrCDM'), exist_ok=True)

## Save adata object

In [822]:
adata.write_h5ad(os.path.join(input_dir, 'mouse_'+mouse+'_combined_fullpreprocessed_CDMthr.h5ad'))